In [2]:
import xgboost as xgb
from read_files import X,y
from sklearn.model_selection import train_test_split
from classifier_methods import *
from sklearn.metrics import accuracy_score

/Users/yananikolova/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Load XML files complete, number of tweet profiles:  420


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [4]:
X_train_features, *settings = get_features_train(X_train)

Generating Train Features
Generating Train Features
Processing features: pos_counts


100%|██████████| 315/315 [00:55<00:00,  5.70it/s]


Individual Predict Processing for features: pos_counts, is complete!
Processing features: author_style_counts


100%|██████████| 315/315 [00:15<00:00, 20.52it/s]


Individual Predict Processing for features: author_style_counts, is complete!
Processing features: lix_score


...

Emoji TF_IDF Processing for features: tf_idf, is complete!
Generating IDF Vector
Processing features: tf_idf


100%|██████████| 315/315 [00:07<00:00, 43.09it/s]


Profanity TF_IDF Processing for features: tf_idf, is complete!
Generating IDF Vector
Processing features: tf_idf


100%|██████████| 315/315 [00:17<00:00, 18.53it/s]


Words TF_IDF Processing for features: tf_idf, is complete!
Emoji Explained Variance:  0.42238571769701533
Profanity Explained Variance:  0.5723355082808844
Word Explained Variance:  0.4686291396061584


In [13]:
X_test_features = get_features_test(X_test, *settings)

Generating Test Features
Processing features: pos_counts


100%|██████████| 105/105 [00:19<00:00,  5.33it/s]


Individual Predict Processing for features: pos_counts, is complete!
Processing features: author_style_counts


100%|██████████| 105/105 [00:05<00:00, 19.58it/s]


Individual Predict Processing for features: author_style_counts, is complete!
Processing features: lix_score


...

Individual Predict Processing for features: tf_idf, is complete!
Processing features: tf_idf


100%|██████████| 105/105 [00:02<00:00, 43.46it/s]


Individual Predict Processing for features: tf_idf, is complete!
Processing features: tf_idf


100%|██████████| 105/105 [00:05<00:00, 18.27it/s]

Words TF_IDF Processing for features: tf_idf, is complete!


In [14]:
y_train[y_train == 'I'] = 1
y_train[y_train == 'NI'] = 0

y_test[y_test == 'I'] = 1
y_test[y_test == 'NI'] = 0

In [15]:
dtrain = xgb.DMatrix(X_train_features, label=y_train)

In [16]:
param = {'max_depth': 4, 'eta': 1, 'objective': 'binary:hinge', 'eval_metric':'error'}
num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [17]:
dtest = xgb.DMatrix(X_test_features)
ypred = bst.predict(dtest)
np.sum(y_test.astype(int) == ypred)/105


0.8

In [20]:
reg = xgb.XGBClassifier(tree_method="hist", eval_metric = 'error', use_label_encoder=False)
# Fit the model using predictor X and response y.
reg.fit(X_train_features, y_train.astype(int))


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='error', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='hist', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [27]:
from sklearn.model_selection import GridSearchCV

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions != test_labels)
    accuracy = (predictions==test_labels).sum()/len(test_labels)
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy


clf_rfc = xgb.XGBClassifier(eval_metric = 'error', use_label_encoder=False, random_state = 0)

clf_rfc.fit(X_train_features, y_train.astype(int))
base_accuracy = evaluate(clf_rfc, X_test_features, y_test.astype(int))

clf_rfc = xgb.XGBClassifier(eval_metric = 'error', use_label_encoder = False, random_state = 0)

param_grid = {
    'n_estimators': [100, 150, 200],
    'tree_method': ['hist', 'auto', 'exact'],
    'max_depth': [3,5, 7, 10, 15],
    'min_child_weight': [1,3,5,7],
    'learning_rate': [0.1, 0.2, 0.3]
    
}

grid_search = GridSearchCV(estimator = clf_rfc, param_grid = param_grid, cv = 4, n_jobs = -1, verbose = 1)
grid_search.fit(X_train_features, y_train.astype(int))
print(grid_search.best_params_)

best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test_features, y_test.astype(int))

print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))




Model Performance
Average Error: 0.0857 degrees.
Accuracy = 0.91%.
Fitting 4 folds for each of 540 candidates, totalling 2160 fits
{'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 7, 'n_estimators': 150, 'tree_method': 'hist'}
Model Performance
Average Error: 0.0952 degrees.
Accuracy = 0.90%.
Improvement of -1.04%.


In [ ]:
evaluate()